In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [2]:
all_mco_data = pd.read_excel('../all_mco_data.xlsx', header=0, sheet_name='all_MCO_data')
all_mco_data.head()

,YEAR,MONTH,DAY,Full Date,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,bin_arrival_delay,int_arrival_delay,arrival_time_bucket,outlier?,col
0,2015,6,6,2015-06-06,6,DL,1649,N692DL,LAX,MCO,...,0.0,0.0,872.0,0.0,0.0,1,872,2,1.0,DL-6-2
1,2015,12,26,2015-12-26,6,UA,208,N34131,DEN,MCO,...,0.0,0.0,10.0,771.0,0.0,1,781,3,1.0,UA-12-3
2,2015,3,21,2015-03-21,6,AA,1351,N3BNAA,LAX,MCO,...,6.0,0.0,719.0,3.0,0.0,1,728,2,1.0,AA-3-2
3,2015,1,8,2015-01-08,4,DL,1315,N327NW,BOS,MCO,...,0.0,0.0,0.0,0.0,681.0,1,681,3,1.0,DL-1-3
4,2015,11,19,2015-11-19,4,DL,2800,N931DL,JFK,MCO,...,0.0,0.0,653.0,0.0,0.0,1,653,1,1.0,DL-11-1


In [ ]:
all_airlines = all_mco_data['AIRLINE'].unique()
all_origins = all_mco_data['ORIGIN_AIRPORT'].unique()

In [ ]:
# Define feature columns
#NOTE: The following is deprecated
day_of_week_col = tf.feature_column.categorical_column_with_identity('day_of_week', num_buckets=7)
month_col = tf.feature_column.categorical_column_with_identity('month', num_buckets=12)
airline_col = tf.feature_column.categorical_column_with_vocabulary_list('airline', all_airlines)
arrival_time_bucket_col = tf.feature_column.categorical_column_with_identity('arrival_time', num_buckets=3)
origin_col = tf.feature_column.categorical_column_with_vocabulary_list('origin', all_origins)

# Set length
N = len(all_mco_data)

# Set feature list
feature_list = [day_of_week_col, month_col, airline_col, arrival_time_bucket_col, origin_col]

In [ ]:
day_of_week = all_mco_data['DAY_OF_WEEK']
month = all_mco_data['MONTH']
airline = all_mco_data['AIRLINE']
arrival_time_bucket = all_mco_data['arrival_time_bucket']
origin = all_mco_data['ORIGIN_AIRPORT']
arrival_delay = all_mco_data['ARRIVAL_DELAY']

day_of_week_train, day_of_week_test, month_train, month_test, \
airline_train, airline_test, arrival_bucket_train, arrival_bucket_test, \
origin_train, origin_test, delay_train, delay_test = train_test_split(
										day_of_week, month, airline, arrival_time_bucket, origin, arrival_delay,  
										random_state=104, 
										train_size=0.8, 
										shuffle=True) 

def train_data():
	features = {
	'day_of_week': day_of_week_train,
	'month': month_train,
	'airline': airline_train,
	'arrival_time_bucket': arrival_bucket_train,
	'origin': origin_train
	}
	labels = delay_train
	return features, labels

In [ ]:
#Define model
#NOTE: The following is deprecated
model = tf.estimator.BoostedTreesRegressor(
		feature_columns=feature_list,
		n_batches_per_layer=1
		)

In [ ]:
#Train model
model.train(train_data, steps=100)

#Evaluate model
train_eval = model.evaluate(train_data, steps=1)
print(pd.Series(train_eval))
